# ハイブリッド検索について体験してみよう

Knowledge bases for Amazon Bedrock では、ベクトルデータベースが OpenSearch Serverless の場合、 ベクトル検索に加えキーワード検索を行う、ハイブリッド検索をサポートしています。

OpenSearch Serverless の場合、次のクエリオプションが選択できます。

- Default
  - Bedrock 側で、ベクターデータベースの構成に最適な検索方法（Hybrid or Semantic）を決定してクエリを実行
- Hybrid
  - ベクトル検索とキーワード検索の両方を利用してクエリを実行
- Semantic
  - ベクトル検索のみでクエリを実行

この章ではハイブリッド検索を利用して、回答制度の向上を試してみます。

サンプルドキュメントの `all` フォルダには社内の申請に必要なドキュメントを格納しています。それぞれの申請には独自のツールを利用する必要があることが明記されています。

- 人事君：住所変更を伴うための社内ツール
- 労務君：給与口座変更を行うための社内ツール
- 経理君：経費清算を行うための社内ツール

## クエリオプションの選択

クエリオプションの選択は RetrieveAndGenerate API の場合、 `retrievalConfiguration` から選択可能です。ベクトル検索とハイブリッド検索の違いを体験してみましょう。

### ベクトル検索のみ

In [ ]:
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")
prompt = "人事君、労務君、経理君の違いを教えて"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "overrideSearchType": "SEMANTIC" # ベクトル検索
                }
            }
        },
    },
)

print(response["output"]["text"])

### ハイブリッド検索

In [ ]:
response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "overrideSearchType": "HYBRID" # ハイブリッド検索
                }
            }
        },
    },
)

print(response["output"]["text"])

## 日本語プラグインを利用したキーワード検索の精度向上

OpenSearch Serverless は、最初から日本語のキーワード検索に対応していません。そのため、日本語の単語が適切に分割されず、検索結果が正確でない可能性があります。

この問題を解決するために、日本語の文章を適切に分析できる「[kuromoji](https://www.elastic.co/guide/en/elasticsearch/plugins/current/analysis-kuromoji.html)」というプラグインを使います。 kurmoji は日本語の形態素解析（単語の分割や品詞の判別）を行うツールです。

### kuromoji プラグインの利用

kuromoji プラグインを利用するために、 OpenSearch のインデックスとナレッジベースのデータソースを再作成を行います。

[rag/aoss.tf](../terraform/rag/aoss.tf) の 126 行目 と 182 行目から 212 行目のコメントを外します。

ターミナルに戻り Terraform の apply を再度実行します。

#### コピペ用

In [ ]:
cd ~/DevelopersIO-2024-bedrock/terraform/rag/
terraform apply --replace aws_bedrockagent_data_source.this --auto-approve

ナレッジベースとデータソースの再同期を行います。Bedrock コンソールから`ナレッジベース`、 `devio-2024-kb` をクリックします。データソースから `devio-2024-datasource` を選択し、`同期`を行います。

![](../images/03/01.png)

### ハイブリッド検索

ハイブリッド検索を再度実行してみます。検索精度は向上しましたか...？

余裕があれば、プロンプトの内容を変えて質問してみましょう。

In [ ]:
response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "overrideSearchType": "HYBRID" # ハイブリッド検索
                }
            }
        },
    },
)

print(response["output"]["text"])

`Sorry, I am unable to assist you with this request.` と表示される場合は 1,2 分ほどおいて再度実行してみてください

## まとめ

このセクションでは次のことを学びました。

- ベクトルデータベースが Open Search Serverless の場合はハイブリッド検索ができる
  - ベクトル検索、キーワード検索に優劣はないです
- Open Search Serverless の日本語プラグインを利用することで、キーワード検索の精度向上が見込める
  - 日本語プラグインには Kuromoji が提供されている

Next: [メタデータフィルタリングについて体験してみよう](./04.ipynb)